In [ ]:
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import urllib.parse
from bs4 import BeautifulSoup

# Spotify API/Google Console 認証情報
with open("config.json", "r") as f:
    config = json.load(f)

CLIENT_ID = config["CLIENT_ID"]
CLIENT_SECRET = config["CLIENT_SECRET"]
API_KEY = config["API_KEY"]
CX = config["CX"]

#spotifyインスタンスを作成
spotify = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id=CLIENT_ID, #Client ID
	    client_secret=CLIENT_SECRET, #Client Secret
    )
)

In [2]:
def get_track_id(artist, title):
    results = spotify.search(q=f"{artist} {title}", limit=1, type="track")
    for track in results["tracks"]["items"]:
  
        name = track["name"]
        track_id = track["id"]
        url = track["external_urls"]["spotify"]

        # audio_featuresは2024-11-29に廃止された
        # track_info = sp.audio_features(track_id)
        # bpm = track_info[0]["tempo"]
        # print(f"{name} : {bpm}")

        print(f"title: {name}, track_id: {track_id}, url: {url}")
        return track_id


In [3]:
def search_song_url(id):
    """Google Custom Search API を使用して TuneBat.com から URL を検索"""
    
    query = f"{id} site:tunebat.com/info/"
    encoded_query = urllib.parse.quote(query)  # URLエンコード
    url = f"https://www.googleapis.com/customsearch/v1?q={encoded_query}&key={API_KEY}&cx={CX}"
    
    try:
        response = requests.get(url)
        data = response.json()

        print(f"data: {json.dumps(data, indent=4, ensure_ascii=False)}")

        for item in data["items"]:
            title = item.get("title")
            link = item.get("link")
            if title and link:
                print(f"title: {title}, link: {link}")
        
        # エラーチェック
        if "error" in data:
            return f"Error: {data['error']['message']}"
        
        # ✅ 検索結果がある場合、最初の結果のURLを取得
        if "items" in data:
            first_result_url = data["items"][0]["link"]
            return first_result_url
        else:
            return "No URL results found."

    except Exception as e:
        return f"Request failed: {str(e)}"

In [4]:
import requests
import json
import re

def search_song_bpm(song_id):
    query = f"{song_id} site:tunebat.com/info/"
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": API_KEY,
        "cx": CX,
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        print(f"data: {json.dumps(data, indent=4, ensure_ascii=False)}")


        bpm_pattern = re.compile(r"(\d{2,3})\s*BPM", re.IGNORECASE)

        for item in data.get("items", []):
            # snippet と title の両方をチェック
            text_sources = [
                item.get("snippet", ""),
                item.get("title", ""),
                item.get("pagemap", {}).get("metatags", [{}])[0].get("og:description", "")
            ]
            for text in text_sources:
                match = bpm_pattern.search(text)
                if match:
                    bpm = int(match.group(1))
                    print(f"🎵 Found BPM: {bpm}")
                    return bpm

        print("⚠️ BPM情報は見つかりませんでした。")
        return None

    except Exception as e:
        print(f"❌ リクエスト失敗: {e}")
        return None


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_audio_features_from_url(url):
    """SeleniumでTunebatのページから各種オーディオ特徴量を取得し、JSON形式で返す"""
    options = Options()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--window-size=1920,1080')

    chrome_prefs = {
        "profile.managed_default_content_settings.stylesheets": 2,
        "profile.managed_default_content_settings.fonts": 2,
    }
    options.add_experimental_option("prefs", chrome_prefs)

    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        wait = WebDriverWait(driver, 5)

        # タイトル情報
        title_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.ant-typography")))
        title = title_element.text.strip()

        title_data = {"title": title}

        # 上段情報（artist, key, camelot, bpm, duration）
        bpm_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3.ant-typography")))
        bpm_texts = [e.text.strip() for e in bpm_elements if e.text.strip()]

        bpm_keys = ["artist", "key", "camelot", "bpm", "duration"]
        bpm_data = dict(zip(bpm_keys, bpm_texts[:len(bpm_keys)]))

        # 下段情報（popularity, energy, danceability, happiness, acousticness, instrumentalness, liveness, speechiness, loudness）
        d_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.ant-progress-text")))
        d_texts = [e.text.strip() for e in d_elements if e.text.strip()]

        d_keys = [
            "popularity", "energy", "danceability", "happiness",
            "acousticness", "instrumentalness", "liveness", "speechiness", "loudness"
        ]
        d_data = dict(zip(d_keys, d_texts[:len(d_keys)]))

        # 合体して返す
        return {**title_data, **bpm_data, **d_data}

    except Exception as e:
        return {"error": str(e)}
    finally:
        driver.quit()


In [6]:
def get_features(artist, title):
    track_id = get_track_id(artist, title)
    url = search_song_url(track_id)
    features = get_audio_features_from_url(url)
    return features

In [7]:
artist = "Vaundy"
title = "Kaiju no hanauta"

features = get_features(artist, title)
features

title: 怪獣の花唄 - replica -, track_id: 26H7pT0IHTko0AA3A35S73, url: https://open.spotify.com/track/26H7pT0IHTko0AA3A35S73
data: {
    "kind": "customsearch#search",
    "url": {
        "type": "application/json",
        "template": "https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json"
    },
    "queries": {
        "request": [
            {
                "title": "Google Custom Search - 26

{'title': '怪獣の花唄 - replica -',
 'artist': 'Vaundy',
 'key': 'D Major',
 'camelot': '10B',
 'bpm': '150',
 'duration': '3:43',
 'popularity': '54',
 'energy': '95',
 'danceability': '46',
 'happiness': '53',
 'acousticness': '3',
 'instrumentalness': '0',
 'liveness': '22',
 'speechiness': '10',
 'loudness': '-3 dB'}